# Cleansing & Transforming  
Summary of steps:  

-Merge weather attributes by city  
-Convert datetime columns from 'object' type to 'datetime' type  
-Add new column to identify the year (ie. 9999)  
-Add new column to identify the month (ie. 1 to 12)  
-Add new column to identify the hour (ie. 0 to 24)  
-Add new column to identify the time of day (ie. AM1, AM2, PM1, PM2)  
-Add new column to identify the compass direction of wind (ie. N, W, S, E, etc)  


In [1]:
# import the library
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Load the Data

In [2]:
df_c = pd.read_csv('../../Data/Raw/city_attributes.csv')
df_h = pd.read_csv('../../Data/Raw/humidity.csv')
df_p = pd.read_csv('../../Data/Raw/pressure.csv')
df_t = pd.read_csv('../../Data/Raw/temperature.csv')
df_d = pd.read_csv('../../Data/Raw/weather_description.csv')
df_wd = pd.read_csv('../../Data/Raw/wind_direction.csv')
df_ws = pd.read_csv('../../Data/Raw/wind_speed.csv')

# Procedures

In [3]:
def timeday(df):
    result = []
    for row in df:
        if row <= 5:
            #early morning 0:00 to 5:59
            result.append('AM1')  
        elif row <= 11:
            #morning 6:00 to 11:59
            result.append('AM2')
        elif row <= 17:
            #afternoon 12:00 to 17:59
            result.append('PM1')
        elif row <= 23:
            #evening 18:00 to 11:59
            result.append('PM2')
        else:
            result.append('')
    return result

In [ ]:
arr = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW', 'N']
def wind_dir(df):
    result = []
    for row in df:
        #print(row)
        if row >= 0:
            k=(row%360)/22.5
            result.append(arr[round(k)])
        else:
            result.append('')
    return result


In [4]:
def new_columns(df):

#     df['datetime'] = pd.to_datetime(df['datetime'])
#     df.insert(1,'date',df['datetime'].dt.date)
#     df.insert(2,'year',df['datetime'].dt.year)
#     df.insert(3,'month',df['datetime'].dt.month)
#     df.insert(4,'day',df['datetime'].dt.day)
#     df.insert(5,'hour',df['datetime'].dt.hour)
    
#     #determine time of day and add to dataframe
#     df_timeday = timeday(df['hour'])
#     df.insert(6,'time_day',df_timeday)
    
#     #determine wind direction and add to dataframe
#     df_windcompass = wind_dir(df['wind_direction'])
#     df.insert(13,'wind_compass',df_windcompass)
    
    #convert temperature to celsius
    df['temperature']=df['temperature']-273.15
    df.rename(columns={'temperature': 'temp_celsius'}, inplace=True)
    
    return df

# Transform the weather data by city

In [5]:
for row in df_c['City']:
    #get humidity attributes 
    data_h = df_h[['datetime',row]]
    data_h = data_h.rename(columns={row:'humidity'})
    
    #get pressure attributes
    data_p = df_p[['datetime',row]]
    data_p = data_p.rename(columns={row:'pressure'})
                           
    #get temperature attributes
    data_t = df_t[['datetime',row]]
    data_t = data_t.rename(columns={row:'temperature'})
    #data_t = data_t['temperature'] - 273.15
        
    #get weather description 
    data_d = df_d[['datetime',row]]
    data_d = data_d.rename(columns={row:'description'})
        
    #get wind direction 
    data_wd = df_wd[['datetime',row]]
    data_wd = data_wd.rename(columns={row:'wind_direction'})
        
    #get wind speed 
    data_ws = df_ws[['datetime',row]]
    data_ws = data_ws.rename(columns={row:'wind_speed'})
    
    #merge all weather attributes onto one dataframe
    df=pd.merge(data_h, data_p, on='datetime',how='outer')
    df=pd.merge(df, data_t, on='datetime',how='outer')
    df=pd.merge(df, data_d, on='datetime',how='outer')
    df=pd.merge(df, data_ws, on='datetime',how='outer')
    df=pd.merge(df, data_wd, on='datetime',how='outer')
    
    df.to_csv('../../Data/Processed/' + row + '.csv',index=False)

# Cleanse the city weather data

In [6]:

def num_missing(x):
    return sum(x.isnull())

for row in df_c['City']:
    print(row)
    df_city = pd.read_csv('../../Data/Processed/'+ row + '.csv')
    
    #remove rows where all attributes are null
    df_city=df_city.dropna(subset=['humidity', 'pressure','temperature','description','wind_speed', 'wind_direction'], how='all')
    print ("\nMissing values per column:")
    print (df_city.apply(num_missing,axis=0))

    df_city.bfill(limit=8)
  #  print ("\nMissing values per column:")
  #  print (df_city.apply(num_missing,axis=0))

    df_city=df_city.ffill(limit=8)
    print ("\nMissing values per column:")
    print (df_city.apply(num_missing,axis=0))

    
    #sort by start date
    df_city=new_columns(df_city)
    df_city = df_city.sort_values(by = ['datetime'])
    df_city.to_csv('../../Data/Processed/' + row + '.csv',index=False)
    
    

Vancouver

Missing values per column:
datetime             0
humidity          1033
pressure          3441
temperature          2
description          0
wind_speed           2
wind_direction       2
dtype: int64

Missing values per column:
datetime             0
humidity            70
pressure          1789
temperature          0
description          0
wind_speed           0
wind_direction       0
dtype: int64
Portland

Missing values per column:
datetime            0
humidity          448
pressure            3
temperature         0
description         0
wind_speed          0
wind_direction      0
dtype: int64

Missing values per column:
datetime          0
humidity          8
pressure          0
temperature       0
description       0
wind_speed        0
wind_direction    0
dtype: int64
San Francisco

Missing values per column:
datetime            0
humidity          149
pressure           22
temperature         0
description         0
wind_speed          1
wind_direction      1
dtype

Jacksonville

Missing values per column:
datetime            0
humidity          208
pressure           14
temperature         0
description         0
wind_speed          0
wind_direction      4
dtype: int64

Missing values per column:
datetime           0
humidity          63
pressure           0
temperature        0
description        0
wind_speed         0
wind_direction     0
dtype: int64
Charlotte

Missing values per column:
datetime            0
humidity          588
pressure            2
temperature         2
description         0
wind_speed          1
wind_direction      0
dtype: int64

Missing values per column:
datetime          0
humidity          4
pressure          0
temperature       0
description       0
wind_speed        0
wind_direction    0
dtype: int64
Miami

Missing values per column:
datetime            0
humidity          294
pressure            1
temperature        12
description         0
wind_speed          2
wind_direction      2
dtype: int64

Missing values p

PermissionError: [Errno 13] Permission denied: '../../Data/Processed/Eilat.csv'